In this notebook, the dataset is analyzed for customer churn irritants for top four product families CIJ, TTO, LCM and TIJ
using Cox Regression Model.

In [1]:
#Importing the necessary libraries
import pandas as pd
import sklearn
import lifelines
from sklearn.preprocessing import StandardScaler

In [2]:
# Ensuring all rows and columns are visible
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:,.2f}'.format)

In [3]:
#Reading the dataset
var = pd.read_csv('analysis.csv',index_col = 0)

In [4]:
var.head()

,Site_Level_Price_Index,CUSTOMER_ID,CUSTOMER_SITE_ID,SHORT_VERTICAL,POSTAL_CODE,CUSTOMER_CLASS,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,DUNS_NUMBER,TRX_DATE,TRX_AMT_USD,Margin,SALES_CHANNEL,QUANTITY,ORDER_TYPE,ORDER_NUM,PRODUCT_FAMILY,PRODUCT_MODEL,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Contract_Category,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,TERRITORY_REGION,TRX_YEAR,Recency,Frequency,Num_of_Trxns,Avg_Trxn_Amt,Avg_Margin,Avg_Quantity,Mode_of_Product_Family,Mode_of_Product_Model,Types_of_Product_Family,Types_of_Product_Model,Most_Frequent_Sales_Channel,Most_Frequent_Order_Type,Avg_Price_Index,Tenure,Cluster_Id,churn_prob_BGNBD,Churned_365,Churned_BGNBD
0,0.79,6482,24,GRAPHICS,60085,END USER,Industrial,S,None,144782380,2020-03-20,"1,855.74","1,381.20",Esker,6,STANDARD DOMESTIC,"512,254,720.00",LASER,FUME EXTRACTION,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,2020,24,17.67,4,"3,102.11","2,901.24",12.00,LCM,INK,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,3,0.05,0,0
1,0.79,6482,24,GRAPHICS,60085,END USER,Industrial,S,None,144782380,2020-04-15,552.68,537.02,SFDC_CPQ,2,EQUIPMENT DOMESTIC,"512,260,527.00",LCM,INK,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,2020,24,17.67,4,"3,102.11","2,901.24",12.00,LCM,INK,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,3,0.05,0,0
2,0.79,6482,24,GRAPHICS,60085,END USER,Industrial,S,None,144782380,2020-05-04,"5,000.00","4,843.37",Online,20,STANDARD DOMESTIC,"512,267,173.00",LCM,INK,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,2020,24,17.67,4,"3,102.11","2,901.24",12.00,LCM,INK,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,3,0.05,0,0
3,0.79,6482,24,GRAPHICS,60085,END USER,Industrial,S,None,144782380,2020-05-11,"5,000.00","4,843.37",Copy,20,STANDARD DOMESTIC,"512,268,803.00",LCM,INK,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,2020,24,17.67,4,"3,102.11","2,901.24",12.00,LCM,INK,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,3,0.05,0,0
4,1.40,37,90,CHEMICALS,65802,END USER,Industrial,S,None,43937895,2017-01-03,213.80,129.59,EDI,20,EDI,"511,872,093.00",CIJ,MAKE-UP,57.00,13.00,44.00,Call,53.00,3.03,unknown,unknown,6.00,6.00,"1,003.00",FSMA,0.00,0,MC,2017,50,18.86,98,233.90,172.39,11.37,CIJ,MAKE-UP,1,3,EDI,EDI,1.35,"1,879.00",3,0.02,0,0


In [5]:
#var['Mode_Product_Family_Overall']
var['PRODUCT_FAMILY'].unique()

array(['LASER', 'LCM', 'CIJ', 'TTO', 'TIJ', 'GRAPHICS BA', 'BINARY ARRAY',
       'LPA', 'RAW MATERIAL', 'GRAPHICS'], dtype=object)

In [6]:
var['PRODUCT_FAMILY'].value_counts()

CIJ             314474
LCM              26654
TTO              21202
GRAPHICS BA       7268
TIJ               6027
LPA               3182
BINARY ARRAY       841
LASER              359
GRAPHICS           272
RAW MATERIAL       122
Name: PRODUCT_FAMILY, dtype: int64

In [7]:
#Drop unnecessary columns - columns with high cardinality, redundancy/collinearity
var.drop(columns=['CUSTOMER_SITE_ID',\
                  'DUNS_NUMBER','TRX_YEAR','POSTAL_CODE',\
                  'ORDER_TYPE',\
                  'Mode_of_Product_Family','Mode_of_Product_Model',\
                  'TRX_DATE','ORDER_NUM','Cluster_Id','CUSTOMER_ID',\
                  'churn_prob_BGNBD','Churned_BGNBD','Recency','Avg_Trxn_Amt'\
                 ],inplace=True)

In [8]:
var.shape

(380401, 38)

In [9]:
var.head()

,Site_Level_Price_Index,SHORT_VERTICAL,CUSTOMER_CLASS,TERRITORY_TYPE,SUPPLIES_SEGMENTATION,SUPPLIES_DECLINE_REASONS,TRX_AMT_USD,Margin,SALES_CHANNEL,QUANTITY,PRODUCT_FAMILY,PRODUCT_MODEL,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Most_Frequent_Interaction_Type,Total_Visits,Total_Cases,Max_Case_Origin,Max_Case_Reason,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Contract_Category,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,TERRITORY_REGION,Frequency,Num_of_Trxns,Avg_Margin,Avg_Quantity,Types_of_Product_Family,Types_of_Product_Model,Most_Frequent_Sales_Channel,Most_Frequent_Order_Type,Avg_Price_Index,Tenure,Churned_365
0,0.79,GRAPHICS,END USER,Industrial,S,None,"1,855.74","1,381.20",Esker,6,LASER,FUME EXTRACTION,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,17.67,4,"2,901.24",12.00,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,0
1,0.79,GRAPHICS,END USER,Industrial,S,None,552.68,537.02,SFDC_CPQ,2,LCM,INK,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,17.67,4,"2,901.24",12.00,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,0
2,0.79,GRAPHICS,END USER,Industrial,S,None,"5,000.00","4,843.37",Online,20,LCM,INK,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,17.67,4,"2,901.24",12.00,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,0
3,0.79,GRAPHICS,END USER,Industrial,S,None,"5,000.00","4,843.37",Copy,20,LCM,INK,13.00,7.00,6.00,Call,81.00,1.97,unknown,unknown,5.00,0.00,0.00,No Contract,0.00,0,MW,17.67,4,"2,901.24",12.00,2,2,Copy,STANDARD DOMESTIC,0.79,77.00,0
4,1.40,CHEMICALS,END USER,Industrial,S,None,213.80,129.59,EDI,20,CIJ,MAKE-UP,57.00,13.00,44.00,Call,53.00,3.03,unknown,unknown,6.00,6.00,"1,003.00",FSMA,0.00,0,MC,18.86,98,172.39,11.37,1,3,EDI,EDI,1.35,"1,879.00",0


In [10]:
var = pd.get_dummies(var,columns = ['SHORT_VERTICAL','CUSTOMER_CLASS','TERRITORY_TYPE','SUPPLIES_SEGMENTATION',
                                    'SUPPLIES_DECLINE_REASONS','SALES_CHANNEL','PRODUCT_FAMILY','PRODUCT_MODEL',
                                    'Most_Frequent_Interaction_Type','Max_Case_Origin','Max_Case_Reason',
                                    'Contract_Category','TERRITORY_REGION','Most_Frequent_Sales_Channel',
                                    'Most_Frequent_Order_Type'], drop_first=True)

In [11]:
var.shape

(380401, 157)

In [12]:
# drop_duplicates
var = var.drop_duplicates()
var.shape

(235924, 157)

In [13]:
#Filter dataset on the four major product families - CIJ, TTO, LCM, TIJ

In [14]:
cij = var[var['PRODUCT_FAMILY_CIJ']==1].copy()

In [15]:
tto = var[var['PRODUCT_FAMILY_TTO']==1].copy()

In [16]:
lcm = var[var['PRODUCT_FAMILY_LCM']==1].copy()

In [17]:
tij = var[var['PRODUCT_FAMILY_TIJ']==1].copy()

# Model building for CIJ

In [19]:
cij.reset_index(inplace=True,drop=True)

In [20]:
cij.head()

,Site_Level_Price_Index,TRX_AMT_USD,Margin,QUANTITY,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Total_Visits,Total_Cases,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,Frequency,Num_of_Trxns,Avg_Margin,Avg_Quantity,Types_of_Product_Family,Types_of_Product_Model,Avg_Price_Index,Tenure,Churned_365,SHORT_VERTICAL_BAKED GOODS & CEREALS,SHORT_VERTICAL_BEVERAGE,SHORT_VERTICAL_BUILDING MATERIALS,SHORT_VERTICAL_CANDY & CONFECTION,SHORT_VERTICAL_CHEMICALS,SHORT_VERTICAL_COSMETICS / PERSONAL CARE,SHORT_VERTICAL_DAIRY & EGGS,SHORT_VERTICAL_DISTRIBUTOR,SHORT_VERTICAL_ELECTRICAL / ELECTRONICS,SHORT_VERTICAL_EXTRUSION / WIRE & CABLE,SHORT_VERTICAL_FISH & SEAFOOD,SHORT_VERTICAL_FROZEN PREPARED MEALS,SHORT_VERTICAL_FRUIT & VEGETABLE,SHORT_VERTICAL_GRAPHICS,SHORT_VERTICAL_INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_MEAT & POULTRY,SHORT_VERTICAL_OEM-INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_OTHER,SHORT_VERTICAL_OTHER FOOD,SHORT_VERTICAL_PACKAGING MATERIALS,SHORT_VERTICAL_PET FOOD & ANIMAL FEED,SHORT_VERTICAL_PHARMA & MEDICAL,SHORT_VERTICAL_POSTAL,SHORT_VERTICAL_SALTY SNACKS,SHORT_VERTICAL_TEXTILE,SHORT_VERTICAL_TOBACCO,SHORT_VERTICAL_UNKNOWN,CUSTOMER_CLASS_END USER,CUSTOMER_CLASS_OEM,TERRITORY_TYPE_Industrial,TERRITORY_TYPE_Postal,SUPPLIES_SEGMENTATION_M,SUPPLIES_SEGMENTATION_S,SUPPLIES_SEGMENTATION_Unclass,SUPPLIES_SEGMENTATION_XL,SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold,SUPPLIES_DECLINE_REASONS_Migration to 1000 Line/TIJ/TTO/LCM/LPA,SUPPLIES_DECLINE_REASONS_Migration to Lasers,SUPPLIES_DECLINE_REASONS_Moved Equipment,SUPPLIES_DECLINE_REASONS_No More Coding Requirement,SUPPLIES_DECLINE_REASONS_None,SUPPLIES_DECLINE_REASONS_Off Brand,SUPPLIES_DECLINE_REASONS_Over Stocked / Timing,SUPPLIES_DECLINE_REASONS_Pricing / Discounting,SUPPLIES_DECLINE_REASONS_Printing/EQ downtime Issues,SUPPLIES_DECLINE_REASONS_Production / Code Reduction,SUPPLIES_DECLINE_REASONS_Production Down (timing),SUPPLIES_DECLINE_REASONS_Project Based,SUPPLIES_DECLINE_REASONS_Recent Regain/Win-back,SUPPLIES_DECLINE_REASONS_Seasonal Producer,SUPPLIES_DECLINE_REASONS_Served by Authorized Distributor,SUPPLIES_DECLINE_REASONS_Site Closed,SUPPLIES_DECLINE_REASONS_VJ Operations Issues,SALES_CHANNEL_Copy,SALES_CHANNEL_ECOMM PO IMPORT,SALES_CHANNEL_EDI,SALES_CHANNEL_Esker,SALES_CHANNEL_IStore Account,SALES_CHANNEL_OCC,SALES_CHANNEL_Online,SALES_CHANNEL_SFDC_CPQ,SALES_CHANNEL_Service Billing,PRODUCT_FAMILY_CIJ,PRODUCT_FAMILY_GRAPHICS,PRODUCT_FAMILY_GRAPHICS BA,PRODUCT_FAMILY_LASER,PRODUCT_FAMILY_LCM,PRODUCT_FAMILY_LPA,PRODUCT_FAMILY_RAW MATERIAL,PRODUCT_FAMILY_TIJ,PRODUCT_FAMILY_TTO,PRODUCT_MODEL_CLEANING SOLUTION,PRODUCT_MODEL_FUME EXTRACTION,PRODUCT_MODEL_INK,PRODUCT_MODEL_LABELS,PRODUCT_MODEL_MAKE-UP,PRODUCT_MODEL_PACKAGING,PRODUCT_MODEL_RIBBONS,PRODUCT_MODEL_SOLVENT,PRODUCT_MODEL_VALUE PACK,Most_Frequent_Interaction_Type_Callback,Most_Frequent_Interaction_Type_Contact Customer,Most_Frequent_Interaction_Type_Customer Meeting,Most_Frequent_Interaction_Type_Dial,Most_Frequent_Interaction_Type_Email,Most_Frequent_Interaction_Type_Make Qualified Sales Call,Most_Frequent_Interaction_Type_Meeting,Most_Frequent_Interaction_Type_Other,Most_Frequent_Interaction_Type_TS Task,Max_Case_Origin_CX Survey Detractor,Max_Case_Origin_Email,Max_Case_Origin_Email - VTI CC Sales Escalations,Max_Case_Origin_Email - VTI NACC,Max_Case_Origin_Email/Fax - VTI CS,Max_Case_Origin_FS Survey Followup,Max_Case_Origin_Install Complete,Max_Case_Origin_Phone,Max_Case_Origin_TS Survey Followup,Max_Case_Origin_unknown,Max_Case_Reason_CX: Customer Care,Max_Case_Reason_CX: Field Sales,Max_Case_Reason_CX: Field Service,Max_Case_Reason_CX: Manufacturing,Max_Case_Reason_CX: Other Team,Max_Case_Reason_CX: Tech Support,Max_Case_Reason_Customer Experience,Max_Case_Reason_unknown,Contract_Category_Full Care,Contract_Category_No Contract,Contract_Category_Supportive,Contract_Category_WFC,TERRITORY_REGION_MW,TERRITORY_REGION_NE,TERRITORY_REGION_NW,TERRITORY_

In [21]:
cijChurned = cij[['Churned_365']]

In [22]:
cijChurned.head()

,Churned_365
0,0
1,0
2,0
3,0
4,0


In [23]:
cij.drop(columns=['Churned_365'],axis=1,inplace=True)

In [24]:
#Scaling
# Instantiate
scaler = StandardScaler()

# fit_transform
cij_scaled = scaler.fit_transform(cij)

In [25]:
cij_scaled = pd.DataFrame(cij_scaled)
cij_scaled.columns = cij.columns
cij_scaled.head()

,Site_Level_Price_Index,TRX_AMT_USD,Margin,QUANTITY,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Total_Visits,Total_Cases,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,Frequency,Num_of_Trxns,Avg_Margin,Avg_Quantity,Types_of_Product_Family,Types_of_Product_Model,Avg_Price_Index,Tenure,SHORT_VERTICAL_BAKED GOODS & CEREALS,SHORT_VERTICAL_BEVERAGE,SHORT_VERTICAL_BUILDING MATERIALS,SHORT_VERTICAL_CANDY & CONFECTION,SHORT_VERTICAL_CHEMICALS,SHORT_VERTICAL_COSMETICS / PERSONAL CARE,SHORT_VERTICAL_DAIRY & EGGS,SHORT_VERTICAL_DISTRIBUTOR,SHORT_VERTICAL_ELECTRICAL / ELECTRONICS,SHORT_VERTICAL_EXTRUSION / WIRE & CABLE,SHORT_VERTICAL_FISH & SEAFOOD,SHORT_VERTICAL_FROZEN PREPARED MEALS,SHORT_VERTICAL_FRUIT & VEGETABLE,SHORT_VERTICAL_GRAPHICS,SHORT_VERTICAL_INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_MEAT & POULTRY,SHORT_VERTICAL_OEM-INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_OTHER,SHORT_VERTICAL_OTHER FOOD,SHORT_VERTICAL_PACKAGING MATERIALS,SHORT_VERTICAL_PET FOOD & ANIMAL FEED,SHORT_VERTICAL_PHARMA & MEDICAL,SHORT_VERTICAL_POSTAL,SHORT_VERTICAL_SALTY SNACKS,SHORT_VERTICAL_TEXTILE,SHORT_VERTICAL_TOBACCO,SHORT_VERTICAL_UNKNOWN,CUSTOMER_CLASS_END USER,CUSTOMER_CLASS_OEM,TERRITORY_TYPE_Industrial,TERRITORY_TYPE_Postal,SUPPLIES_SEGMENTATION_M,SUPPLIES_SEGMENTATION_S,SUPPLIES_SEGMENTATION_Unclass,SUPPLIES_SEGMENTATION_XL,SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold,SUPPLIES_DECLINE_REASONS_Migration to 1000 Line/TIJ/TTO/LCM/LPA,SUPPLIES_DECLINE_REASONS_Migration to Lasers,SUPPLIES_DECLINE_REASONS_Moved Equipment,SUPPLIES_DECLINE_REASONS_No More Coding Requirement,SUPPLIES_DECLINE_REASONS_None,SUPPLIES_DECLINE_REASONS_Off Brand,SUPPLIES_DECLINE_REASONS_Over Stocked / Timing,SUPPLIES_DECLINE_REASONS_Pricing / Discounting,SUPPLIES_DECLINE_REASONS_Printing/EQ downtime Issues,SUPPLIES_DECLINE_REASONS_Production / Code Reduction,SUPPLIES_DECLINE_REASONS_Production Down (timing),SUPPLIES_DECLINE_REASONS_Project Based,SUPPLIES_DECLINE_REASONS_Recent Regain/Win-back,SUPPLIES_DECLINE_REASONS_Seasonal Producer,SUPPLIES_DECLINE_REASONS_Served by Authorized Distributor,SUPPLIES_DECLINE_REASONS_Site Closed,SUPPLIES_DECLINE_REASONS_VJ Operations Issues,SALES_CHANNEL_Copy,SALES_CHANNEL_ECOMM PO IMPORT,SALES_CHANNEL_EDI,SALES_CHANNEL_Esker,SALES_CHANNEL_IStore Account,SALES_CHANNEL_OCC,SALES_CHANNEL_Online,SALES_CHANNEL_SFDC_CPQ,SALES_CHANNEL_Service Billing,PRODUCT_FAMILY_CIJ,PRODUCT_FAMILY_GRAPHICS,PRODUCT_FAMILY_GRAPHICS BA,PRODUCT_FAMILY_LASER,PRODUCT_FAMILY_LCM,PRODUCT_FAMILY_LPA,PRODUCT_FAMILY_RAW MATERIAL,PRODUCT_FAMILY_TIJ,PRODUCT_FAMILY_TTO,PRODUCT_MODEL_CLEANING SOLUTION,PRODUCT_MODEL_FUME EXTRACTION,PRODUCT_MODEL_INK,PRODUCT_MODEL_LABELS,PRODUCT_MODEL_MAKE-UP,PRODUCT_MODEL_PACKAGING,PRODUCT_MODEL_RIBBONS,PRODUCT_MODEL_SOLVENT,PRODUCT_MODEL_VALUE PACK,Most_Frequent_Interaction_Type_Callback,Most_Frequent_Interaction_Type_Contact Customer,Most_Frequent_Interaction_Type_Customer Meeting,Most_Frequent_Interaction_Type_Dial,Most_Frequent_Interaction_Type_Email,Most_Frequent_Interaction_Type_Make Qualified Sales Call,Most_Frequent_Interaction_Type_Meeting,Most_Frequent_Interaction_Type_Other,Most_Frequent_Interaction_Type_TS Task,Max_Case_Origin_CX Survey Detractor,Max_Case_Origin_Email,Max_Case_Origin_Email - VTI CC Sales Escalations,Max_Case_Origin_Email - VTI NACC,Max_Case_Origin_Email/Fax - VTI CS,Max_Case_Origin_FS Survey Followup,Max_Case_Origin_Install Complete,Max_Case_Origin_Phone,Max_Case_Origin_TS Survey Followup,Max_Case_Origin_unknown,Max_Case_Reason_CX: Customer Care,Max_Case_Reason_CX: Field Sales,Max_Case_Reason_CX: Field Service,Max_Case_Reason_CX: Manufacturing,Max_Case_Reason_CX: Other Team,Max_Case_Reason_CX: Tech Support,Max_Case_Reason_Customer Experience,Max_Case_Reason_unknown,Contract_Category_Full Care,Contract_Category_No Contract,Contract_Category_Supportive,Contract_Category_WFC,TERRITORY_REGION_MW,TERRITORY_REGION_NE,TERRITORY_REGION_NW,TERRITORY_REGION_SC,TE

In [26]:
cij_final = pd.concat([cij_scaled,cijChurned],axis=1,sort=False)

In [27]:
cij_final.head()

,Site_Level_Price_Index,TRX_AMT_USD,Margin,QUANTITY,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Total_Visits,Total_Cases,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,Frequency,Num_of_Trxns,Avg_Margin,Avg_Quantity,Types_of_Product_Family,Types_of_Product_Model,Avg_Price_Index,Tenure,SHORT_VERTICAL_BAKED GOODS & CEREALS,SHORT_VERTICAL_BEVERAGE,SHORT_VERTICAL_BUILDING MATERIALS,SHORT_VERTICAL_CANDY & CONFECTION,SHORT_VERTICAL_CHEMICALS,SHORT_VERTICAL_COSMETICS / PERSONAL CARE,SHORT_VERTICAL_DAIRY & EGGS,SHORT_VERTICAL_DISTRIBUTOR,SHORT_VERTICAL_ELECTRICAL / ELECTRONICS,SHORT_VERTICAL_EXTRUSION / WIRE & CABLE,SHORT_VERTICAL_FISH & SEAFOOD,SHORT_VERTICAL_FROZEN PREPARED MEALS,SHORT_VERTICAL_FRUIT & VEGETABLE,SHORT_VERTICAL_GRAPHICS,SHORT_VERTICAL_INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_MEAT & POULTRY,SHORT_VERTICAL_OEM-INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_OTHER,SHORT_VERTICAL_OTHER FOOD,SHORT_VERTICAL_PACKAGING MATERIALS,SHORT_VERTICAL_PET FOOD & ANIMAL FEED,SHORT_VERTICAL_PHARMA & MEDICAL,SHORT_VERTICAL_POSTAL,SHORT_VERTICAL_SALTY SNACKS,SHORT_VERTICAL_TEXTILE,SHORT_VERTICAL_TOBACCO,SHORT_VERTICAL_UNKNOWN,CUSTOMER_CLASS_END USER,CUSTOMER_CLASS_OEM,TERRITORY_TYPE_Industrial,TERRITORY_TYPE_Postal,SUPPLIES_SEGMENTATION_M,SUPPLIES_SEGMENTATION_S,SUPPLIES_SEGMENTATION_Unclass,SUPPLIES_SEGMENTATION_XL,SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold,SUPPLIES_DECLINE_REASONS_Migration to 1000 Line/TIJ/TTO/LCM/LPA,SUPPLIES_DECLINE_REASONS_Migration to Lasers,SUPPLIES_DECLINE_REASONS_Moved Equipment,SUPPLIES_DECLINE_REASONS_No More Coding Requirement,SUPPLIES_DECLINE_REASONS_None,SUPPLIES_DECLINE_REASONS_Off Brand,SUPPLIES_DECLINE_REASONS_Over Stocked / Timing,SUPPLIES_DECLINE_REASONS_Pricing / Discounting,SUPPLIES_DECLINE_REASONS_Printing/EQ downtime Issues,SUPPLIES_DECLINE_REASONS_Production / Code Reduction,SUPPLIES_DECLINE_REASONS_Production Down (timing),SUPPLIES_DECLINE_REASONS_Project Based,SUPPLIES_DECLINE_REASONS_Recent Regain/Win-back,SUPPLIES_DECLINE_REASONS_Seasonal Producer,SUPPLIES_DECLINE_REASONS_Served by Authorized Distributor,SUPPLIES_DECLINE_REASONS_Site Closed,SUPPLIES_DECLINE_REASONS_VJ Operations Issues,SALES_CHANNEL_Copy,SALES_CHANNEL_ECOMM PO IMPORT,SALES_CHANNEL_EDI,SALES_CHANNEL_Esker,SALES_CHANNEL_IStore Account,SALES_CHANNEL_OCC,SALES_CHANNEL_Online,SALES_CHANNEL_SFDC_CPQ,SALES_CHANNEL_Service Billing,PRODUCT_FAMILY_CIJ,PRODUCT_FAMILY_GRAPHICS,PRODUCT_FAMILY_GRAPHICS BA,PRODUCT_FAMILY_LASER,PRODUCT_FAMILY_LCM,PRODUCT_FAMILY_LPA,PRODUCT_FAMILY_RAW MATERIAL,PRODUCT_FAMILY_TIJ,PRODUCT_FAMILY_TTO,PRODUCT_MODEL_CLEANING SOLUTION,PRODUCT_MODEL_FUME EXTRACTION,PRODUCT_MODEL_INK,PRODUCT_MODEL_LABELS,PRODUCT_MODEL_MAKE-UP,PRODUCT_MODEL_PACKAGING,PRODUCT_MODEL_RIBBONS,PRODUCT_MODEL_SOLVENT,PRODUCT_MODEL_VALUE PACK,Most_Frequent_Interaction_Type_Callback,Most_Frequent_Interaction_Type_Contact Customer,Most_Frequent_Interaction_Type_Customer Meeting,Most_Frequent_Interaction_Type_Dial,Most_Frequent_Interaction_Type_Email,Most_Frequent_Interaction_Type_Make Qualified Sales Call,Most_Frequent_Interaction_Type_Meeting,Most_Frequent_Interaction_Type_Other,Most_Frequent_Interaction_Type_TS Task,Max_Case_Origin_CX Survey Detractor,Max_Case_Origin_Email,Max_Case_Origin_Email - VTI CC Sales Escalations,Max_Case_Origin_Email - VTI NACC,Max_Case_Origin_Email/Fax - VTI CS,Max_Case_Origin_FS Survey Followup,Max_Case_Origin_Install Complete,Max_Case_Origin_Phone,Max_Case_Origin_TS Survey Followup,Max_Case_Origin_unknown,Max_Case_Reason_CX: Customer Care,Max_Case_Reason_CX: Field Sales,Max_Case_Reason_CX: Field Service,Max_Case_Reason_CX: Manufacturing,Max_Case_Reason_CX: Other Team,Max_Case_Reason_CX: Tech Support,Max_Case_Reason_Customer Experience,Max_Case_Reason_unknown,Contract_Category_Full Care,Contract_Category_No Contract,Contract_Category_Supportive,Contract_Category_WFC,TERRITORY_REGION_MW,TERRITORY_REGION_NE,TERRITORY_REGION_NW,TERRITORY_REGION_SC,TE

In [28]:
cij_final = cij_final.drop(columns=['SALES_CHANNEL_Service Billing', 'PRODUCT_FAMILY_CIJ', 'PRODUCT_FAMILY_GRAPHICS', 'PRODUCT_FAMILY_GRAPHICS BA', 'PRODUCT_FAMILY_LASER', 'PRODUCT_FAMILY_LCM', 'PRODUCT_FAMILY_LPA', 'PRODUCT_FAMILY_RAW MATERIAL', 'PRODUCT_FAMILY_TIJ', 'PRODUCT_FAMILY_TTO', 'PRODUCT_MODEL_FUME EXTRACTION', 'PRODUCT_MODEL_LABELS', 'PRODUCT_MODEL_PACKAGING', 'PRODUCT_MODEL_RIBBONS', 'PRODUCT_MODEL_SOLVENT', 'Most_Frequent_Interaction_Type_Make Qualified Sales Call', 'Most_Frequent_Sales_Channel_Service Billing', 'Most_Frequent_Order_Type_SERVICE'],axis=1)

In [29]:
cij_final.shape

(196766, 139)

In [30]:
cph = lifelines.CoxPHFitter(penalizer=0.00001)
cph.fit(cij_final, step_size=0.05, duration_col='Tenure', event_col='Churned_365', show_progress=False)
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 196766 total observations, 178826 right-censored observations>
             duration col = 'Tenure'
                event col = 'Churned_365'
                penalizer = 1e-05
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 196766
number of events observed = 17940
   partial log-likelihood = -187683.53
         time fit was run = 2020-08-05 04:41:24 UTC

---
                                                                  coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                               
Site_Level_Price_Index                                            0.00       1.00       0.01            -0.02             0.03                 0.98                 1.03
TRX_AMT_USD                                                       0.36       1.43       0.05             0.26             0.46                 1.29                 1.58
Margin                                                           -0.40       0.67       0.06            -0.51            -0.29                 0.60                 0.75
QUANTITY                                                          0.01       1.01       0.01            -0.02             0.04                 0.98                 1.04
Total_SVC_Incidents                                               0.11       1.11       0.56            -1.00             1.21                 0.37                 3.36
Total_Repeat_Calls                                                0.23       1.26       0.20            -0.16             0.62                 0.85                 1.86
Total_FTF_Calls                                                   0.04       1.04       0.39            -0.73             0.81                 0.48                 2.24
Total_Visits                                                     -0.89       0.41       0.03            -0.94            -0.84                 0.39                 0.43
Total_Cases                                                       0.21       1.23       0.01             0.18             0.24                 1.20                 1.27
Num_of_Active_Install_Bases                                      -0.11       0.90       0.03            -0.16            -0.06                 0.85                 0.94
Total_Contracts                                                  -0.07       0.93       0.03            -0.13            -0.02                 0.88                 0.98
Contract_length                                                  -0.44       0.65       0.02            -0.47            -0.40                 0.63                 0.67
Num_of_Inactive_Install_Bases                                    -0.07       0.93       0.01            -0.10            -0.05                 0.90                 0.96
STRATEGIC_ACCOUNTS                                                0.18       1.20       0.01             0.16             0.21                 1.17                 1.23
Frequency                                                        -0.40       0.67       0.01            -0.42            -0.38                 0.65                 0.68
Num_of_Trxns                                                     -2.06       0.13       0.05            -2.15            -1.97                 0.12                 0.14
Avg_Margin                                                        0.12       1.13       0.02             0.09             0.15                 1.09                 1.17
Avg_Quantity                                                     -0.00       1.00       0.02            -0.04             0.03                 0.96                 1.03
Types_of_Product_Family                                           0.23       1.26       0.01             0.20             0.25                 1.23                 1.29
Types_

In [31]:
cij_final_table = cph.summary[cph.summary['p']<0.005].sort_values(by='exp(coef)', ascending=False).head(10)

# Model building for TTO

In [33]:
tto.reset_index(inplace=True,drop=True)

In [34]:
tto.head()

,Site_Level_Price_Index,TRX_AMT_USD,Margin,QUANTITY,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Total_Visits,Total_Cases,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,Frequency,Num_of_Trxns,Avg_Margin,Avg_Quantity,Types_of_Product_Family,Types_of_Product_Model,Avg_Price_Index,Tenure,Churned_365,SHORT_VERTICAL_BAKED GOODS & CEREALS,SHORT_VERTICAL_BEVERAGE,SHORT_VERTICAL_BUILDING MATERIALS,SHORT_VERTICAL_CANDY & CONFECTION,SHORT_VERTICAL_CHEMICALS,SHORT_VERTICAL_COSMETICS / PERSONAL CARE,SHORT_VERTICAL_DAIRY & EGGS,SHORT_VERTICAL_DISTRIBUTOR,SHORT_VERTICAL_ELECTRICAL / ELECTRONICS,SHORT_VERTICAL_EXTRUSION / WIRE & CABLE,SHORT_VERTICAL_FISH & SEAFOOD,SHORT_VERTICAL_FROZEN PREPARED MEALS,SHORT_VERTICAL_FRUIT & VEGETABLE,SHORT_VERTICAL_GRAPHICS,SHORT_VERTICAL_INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_MEAT & POULTRY,SHORT_VERTICAL_OEM-INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_OTHER,SHORT_VERTICAL_OTHER FOOD,SHORT_VERTICAL_PACKAGING MATERIALS,SHORT_VERTICAL_PET FOOD & ANIMAL FEED,SHORT_VERTICAL_PHARMA & MEDICAL,SHORT_VERTICAL_POSTAL,SHORT_VERTICAL_SALTY SNACKS,SHORT_VERTICAL_TEXTILE,SHORT_VERTICAL_TOBACCO,SHORT_VERTICAL_UNKNOWN,CUSTOMER_CLASS_END USER,CUSTOMER_CLASS_OEM,TERRITORY_TYPE_Industrial,TERRITORY_TYPE_Postal,SUPPLIES_SEGMENTATION_M,SUPPLIES_SEGMENTATION_S,SUPPLIES_SEGMENTATION_Unclass,SUPPLIES_SEGMENTATION_XL,SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold,SUPPLIES_DECLINE_REASONS_Migration to 1000 Line/TIJ/TTO/LCM/LPA,SUPPLIES_DECLINE_REASONS_Migration to Lasers,SUPPLIES_DECLINE_REASONS_Moved Equipment,SUPPLIES_DECLINE_REASONS_No More Coding Requirement,SUPPLIES_DECLINE_REASONS_None,SUPPLIES_DECLINE_REASONS_Off Brand,SUPPLIES_DECLINE_REASONS_Over Stocked / Timing,SUPPLIES_DECLINE_REASONS_Pricing / Discounting,SUPPLIES_DECLINE_REASONS_Printing/EQ downtime Issues,SUPPLIES_DECLINE_REASONS_Production / Code Reduction,SUPPLIES_DECLINE_REASONS_Production Down (timing),SUPPLIES_DECLINE_REASONS_Project Based,SUPPLIES_DECLINE_REASONS_Recent Regain/Win-back,SUPPLIES_DECLINE_REASONS_Seasonal Producer,SUPPLIES_DECLINE_REASONS_Served by Authorized Distributor,SUPPLIES_DECLINE_REASONS_Site Closed,SUPPLIES_DECLINE_REASONS_VJ Operations Issues,SALES_CHANNEL_Copy,SALES_CHANNEL_ECOMM PO IMPORT,SALES_CHANNEL_EDI,SALES_CHANNEL_Esker,SALES_CHANNEL_IStore Account,SALES_CHANNEL_OCC,SALES_CHANNEL_Online,SALES_CHANNEL_SFDC_CPQ,SALES_CHANNEL_Service Billing,PRODUCT_FAMILY_CIJ,PRODUCT_FAMILY_GRAPHICS,PRODUCT_FAMILY_GRAPHICS BA,PRODUCT_FAMILY_LASER,PRODUCT_FAMILY_LCM,PRODUCT_FAMILY_LPA,PRODUCT_FAMILY_RAW MATERIAL,PRODUCT_FAMILY_TIJ,PRODUCT_FAMILY_TTO,PRODUCT_MODEL_CLEANING SOLUTION,PRODUCT_MODEL_FUME EXTRACTION,PRODUCT_MODEL_INK,PRODUCT_MODEL_LABELS,PRODUCT_MODEL_MAKE-UP,PRODUCT_MODEL_PACKAGING,PRODUCT_MODEL_RIBBONS,PRODUCT_MODEL_SOLVENT,PRODUCT_MODEL_VALUE PACK,Most_Frequent_Interaction_Type_Callback,Most_Frequent_Interaction_Type_Contact Customer,Most_Frequent_Interaction_Type_Customer Meeting,Most_Frequent_Interaction_Type_Dial,Most_Frequent_Interaction_Type_Email,Most_Frequent_Interaction_Type_Make Qualified Sales Call,Most_Frequent_Interaction_Type_Meeting,Most_Frequent_Interaction_Type_Other,Most_Frequent_Interaction_Type_TS Task,Max_Case_Origin_CX Survey Detractor,Max_Case_Origin_Email,Max_Case_Origin_Email - VTI CC Sales Escalations,Max_Case_Origin_Email - VTI NACC,Max_Case_Origin_Email/Fax - VTI CS,Max_Case_Origin_FS Survey Followup,Max_Case_Origin_Install Complete,Max_Case_Origin_Phone,Max_Case_Origin_TS Survey Followup,Max_Case_Origin_unknown,Max_Case_Reason_CX: Customer Care,Max_Case_Reason_CX: Field Sales,Max_Case_Reason_CX: Field Service,Max_Case_Reason_CX: Manufacturing,Max_Case_Reason_CX: Other Team,Max_Case_Reason_CX: Tech Support,Max_Case_Reason_Customer Experience,Max_Case_Reason_unknown,Contract_Category_Full Care,Contract_Category_No Contract,Contract_Category_Supportive,Contract_Category_WFC,TERRITORY_REGION_MW,TERRITORY_REGION_NE,TERRITORY_REGION_NW,TERRITORY_

In [35]:
tto.shape

(12346, 157)

In [36]:
ttoChurned = tto[['Churned_365']]

In [37]:
tto.drop(columns=['Churned_365'],axis=1,inplace=True)

In [38]:
#Scaling
# Instantiate
scaler = StandardScaler()

# fit_transform
tto_scaled = scaler.fit_transform(tto)

In [39]:
tto_scaled = pd.DataFrame(tto_scaled)
tto_scaled.columns = tto.columns
tto_scaled.head()

,Site_Level_Price_Index,TRX_AMT_USD,Margin,QUANTITY,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Total_Visits,Total_Cases,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,Frequency,Num_of_Trxns,Avg_Margin,Avg_Quantity,Types_of_Product_Family,Types_of_Product_Model,Avg_Price_Index,Tenure,SHORT_VERTICAL_BAKED GOODS & CEREALS,SHORT_VERTICAL_BEVERAGE,SHORT_VERTICAL_BUILDING MATERIALS,SHORT_VERTICAL_CANDY & CONFECTION,SHORT_VERTICAL_CHEMICALS,SHORT_VERTICAL_COSMETICS / PERSONAL CARE,SHORT_VERTICAL_DAIRY & EGGS,SHORT_VERTICAL_DISTRIBUTOR,SHORT_VERTICAL_ELECTRICAL / ELECTRONICS,SHORT_VERTICAL_EXTRUSION / WIRE & CABLE,SHORT_VERTICAL_FISH & SEAFOOD,SHORT_VERTICAL_FROZEN PREPARED MEALS,SHORT_VERTICAL_FRUIT & VEGETABLE,SHORT_VERTICAL_GRAPHICS,SHORT_VERTICAL_INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_MEAT & POULTRY,SHORT_VERTICAL_OEM-INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_OTHER,SHORT_VERTICAL_OTHER FOOD,SHORT_VERTICAL_PACKAGING MATERIALS,SHORT_VERTICAL_PET FOOD & ANIMAL FEED,SHORT_VERTICAL_PHARMA & MEDICAL,SHORT_VERTICAL_POSTAL,SHORT_VERTICAL_SALTY SNACKS,SHORT_VERTICAL_TEXTILE,SHORT_VERTICAL_TOBACCO,SHORT_VERTICAL_UNKNOWN,CUSTOMER_CLASS_END USER,CUSTOMER_CLASS_OEM,TERRITORY_TYPE_Industrial,TERRITORY_TYPE_Postal,SUPPLIES_SEGMENTATION_M,SUPPLIES_SEGMENTATION_S,SUPPLIES_SEGMENTATION_Unclass,SUPPLIES_SEGMENTATION_XL,SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold,SUPPLIES_DECLINE_REASONS_Migration to 1000 Line/TIJ/TTO/LCM/LPA,SUPPLIES_DECLINE_REASONS_Migration to Lasers,SUPPLIES_DECLINE_REASONS_Moved Equipment,SUPPLIES_DECLINE_REASONS_No More Coding Requirement,SUPPLIES_DECLINE_REASONS_None,SUPPLIES_DECLINE_REASONS_Off Brand,SUPPLIES_DECLINE_REASONS_Over Stocked / Timing,SUPPLIES_DECLINE_REASONS_Pricing / Discounting,SUPPLIES_DECLINE_REASONS_Printing/EQ downtime Issues,SUPPLIES_DECLINE_REASONS_Production / Code Reduction,SUPPLIES_DECLINE_REASONS_Production Down (timing),SUPPLIES_DECLINE_REASONS_Project Based,SUPPLIES_DECLINE_REASONS_Recent Regain/Win-back,SUPPLIES_DECLINE_REASONS_Seasonal Producer,SUPPLIES_DECLINE_REASONS_Served by Authorized Distributor,SUPPLIES_DECLINE_REASONS_Site Closed,SUPPLIES_DECLINE_REASONS_VJ Operations Issues,SALES_CHANNEL_Copy,SALES_CHANNEL_ECOMM PO IMPORT,SALES_CHANNEL_EDI,SALES_CHANNEL_Esker,SALES_CHANNEL_IStore Account,SALES_CHANNEL_OCC,SALES_CHANNEL_Online,SALES_CHANNEL_SFDC_CPQ,SALES_CHANNEL_Service Billing,PRODUCT_FAMILY_CIJ,PRODUCT_FAMILY_GRAPHICS,PRODUCT_FAMILY_GRAPHICS BA,PRODUCT_FAMILY_LASER,PRODUCT_FAMILY_LCM,PRODUCT_FAMILY_LPA,PRODUCT_FAMILY_RAW MATERIAL,PRODUCT_FAMILY_TIJ,PRODUCT_FAMILY_TTO,PRODUCT_MODEL_CLEANING SOLUTION,PRODUCT_MODEL_FUME EXTRACTION,PRODUCT_MODEL_INK,PRODUCT_MODEL_LABELS,PRODUCT_MODEL_MAKE-UP,PRODUCT_MODEL_PACKAGING,PRODUCT_MODEL_RIBBONS,PRODUCT_MODEL_SOLVENT,PRODUCT_MODEL_VALUE PACK,Most_Frequent_Interaction_Type_Callback,Most_Frequent_Interaction_Type_Contact Customer,Most_Frequent_Interaction_Type_Customer Meeting,Most_Frequent_Interaction_Type_Dial,Most_Frequent_Interaction_Type_Email,Most_Frequent_Interaction_Type_Make Qualified Sales Call,Most_Frequent_Interaction_Type_Meeting,Most_Frequent_Interaction_Type_Other,Most_Frequent_Interaction_Type_TS Task,Max_Case_Origin_CX Survey Detractor,Max_Case_Origin_Email,Max_Case_Origin_Email - VTI CC Sales Escalations,Max_Case_Origin_Email - VTI NACC,Max_Case_Origin_Email/Fax - VTI CS,Max_Case_Origin_FS Survey Followup,Max_Case_Origin_Install Complete,Max_Case_Origin_Phone,Max_Case_Origin_TS Survey Followup,Max_Case_Origin_unknown,Max_Case_Reason_CX: Customer Care,Max_Case_Reason_CX: Field Sales,Max_Case_Reason_CX: Field Service,Max_Case_Reason_CX: Manufacturing,Max_Case_Reason_CX: Other Team,Max_Case_Reason_CX: Tech Support,Max_Case_Reason_Customer Experience,Max_Case_Reason_unknown,Contract_Category_Full Care,Contract_Category_No Contract,Contract_Category_Supportive,Contract_Category_WFC,TERRITORY_REGION_MW,TERRITORY_REGION_NE,TERRITORY_REGION_NW,TERRITORY_REGION_SC,TE

In [40]:
tto_final = pd.concat([tto_scaled,ttoChurned],axis=1,sort=False)

In [41]:
tto_final = tto_final.drop(columns=['SHORT_VERTICAL_POSTAL', 'TERRITORY_TYPE_Postal', 'SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold', 'SALES_CHANNEL_ECOMM PO IMPORT', 'SALES_CHANNEL_Service Billing', 'PRODUCT_FAMILY_CIJ', 'PRODUCT_FAMILY_GRAPHICS', 'PRODUCT_FAMILY_GRAPHICS BA', 'PRODUCT_FAMILY_LASER', 'PRODUCT_FAMILY_LCM', 'PRODUCT_FAMILY_LPA', 'PRODUCT_FAMILY_RAW MATERIAL', 'PRODUCT_FAMILY_TIJ', 'PRODUCT_FAMILY_TTO', 'PRODUCT_MODEL_CLEANING SOLUTION', 'PRODUCT_MODEL_FUME EXTRACTION', 'PRODUCT_MODEL_INK', 'PRODUCT_MODEL_LABELS', 'PRODUCT_MODEL_MAKE-UP', 'PRODUCT_MODEL_PACKAGING', 'PRODUCT_MODEL_RIBBONS', 'PRODUCT_MODEL_SOLVENT', 'PRODUCT_MODEL_VALUE PACK', 'Most_Frequent_Interaction_Type_Callback', 'Most_Frequent_Interaction_Type_Contact Customer', 'Most_Frequent_Interaction_Type_Dial', 'Most_Frequent_Interaction_Type_Make Qualified Sales Call', 'Most_Frequent_Interaction_Type_TS Task', 'Max_Case_Origin_CX Survey Detractor', 'Max_Case_Origin_Email - VTI CC Sales Escalations', 'Max_Case_Reason_CX: Field Sales', 'Max_Case_Reason_CX: Manufacturing', 'Max_Case_Reason_CX: Other Team', 'Contract_Category_Supportive', 'Contract_Category_WFC', 'Most_Frequent_Sales_Channel_Service Billing', 'Most_Frequent_Order_Type_SERVICE', 'Most_Frequent_Order_Type_US FULL CARE INTERNATIONAL'],axis=1)

In [42]:
tto_final.shape

(12346, 119)

In [43]:
cph1 = lifelines.CoxPHFitter(penalizer=0.000001)
cph1.fit(tto_final, step_size=0.05, duration_col='Tenure', event_col='Churned_365', show_progress=False)
cph1.print_summary()

<lifelines.CoxPHFitter: fitted with 12346 total observations, 10409 right-censored observations>
             duration col = 'Tenure'
                event col = 'Churned_365'
                penalizer = 1e-06
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 12346
number of events observed = 1937
   partial log-likelihood = -14507.03
         time fit was run = 2020-08-05 04:48:44 UTC

---
                                                                  coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                               
Site_Level_Price_Index                                           -0.07       0.93       0.04            -0.15             0.01                 0.86                 1.01
TRX_AMT_USD                                                       0.90       2.46       0.13             0.65             1.15                 1.91                 3.16
Margin                                                           -0.74       0.48       0.12            -0.96            -0.51                 0.38                 0.60
QUANTITY                                                         -0.28       0.75       0.07            -0.42            -0.14                 0.65                 0.87
Total_SVC_Incidents                                               0.00       1.00       7.06           -13.84            13.84                 0.00             1.03e+06
Total_Repeat_Calls                                               -0.16       0.85       2.19            -4.46             4.14                 0.01                63.06
Total_FTF_Calls                                                   0.07       1.07       5.13            -9.99            10.13                 0.00             24992.82
Total_Visits                                                     -0.08       0.92       0.06            -0.19             0.03                 0.82                 1.03
Total_Cases                                                       0.23       1.26       0.03             0.17             0.30                 1.18                 1.35
Num_of_Active_Install_Bases                                      -0.13       0.88       0.05            -0.23            -0.02                 0.79                 0.98
Total_Contracts                                                   0.07       1.08       0.07            -0.07             0.22                 0.93                 1.25
Contract_length                                                  -0.19       0.82       0.08            -0.35            -0.04                 0.70                 0.97
Num_of_Inactive_Install_Bases                                    -0.02       0.98       0.04            -0.10             0.06                 0.91                 1.06
STRATEGIC_ACCOUNTS                                                0.44       1.55       0.05             0.35             0.53                 1.41                 1.69
Frequency                                                        -0.08       0.93       0.03            -0.13            -0.02                 0.87                 0.98
Num_of_Trxns                                                     -0.30       0.74       0.07            -0.44            -0.16                 0.65                 0.85
Avg_Margin                                                       -0.15       0.86       0.07            -0.29            -0.01                 0.75                 0.99
Avg_Quantity                                                      0.28       1.32       0.07             0.15             0.41                 1.16                 1.50
Types_of_Product_Family                                          -0.18       0.84       0.07            -0.32            -0.03                 0.72                 0.97
Types_of_Pr

In [44]:
tto_final_table = cph1.summary[cph1.summary['p']<0.005].sort_values(by='exp(coef)', ascending=False).head(10)

# Model building for LCM product family

In [46]:
lcm.reset_index(inplace=True,drop=True)

In [47]:
lcm.head()

,Site_Level_Price_Index,TRX_AMT_USD,Margin,QUANTITY,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Total_Visits,Total_Cases,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,Frequency,Num_of_Trxns,Avg_Margin,Avg_Quantity,Types_of_Product_Family,Types_of_Product_Model,Avg_Price_Index,Tenure,Churned_365,SHORT_VERTICAL_BAKED GOODS & CEREALS,SHORT_VERTICAL_BEVERAGE,SHORT_VERTICAL_BUILDING MATERIALS,SHORT_VERTICAL_CANDY & CONFECTION,SHORT_VERTICAL_CHEMICALS,SHORT_VERTICAL_COSMETICS / PERSONAL CARE,SHORT_VERTICAL_DAIRY & EGGS,SHORT_VERTICAL_DISTRIBUTOR,SHORT_VERTICAL_ELECTRICAL / ELECTRONICS,SHORT_VERTICAL_EXTRUSION / WIRE & CABLE,SHORT_VERTICAL_FISH & SEAFOOD,SHORT_VERTICAL_FROZEN PREPARED MEALS,SHORT_VERTICAL_FRUIT & VEGETABLE,SHORT_VERTICAL_GRAPHICS,SHORT_VERTICAL_INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_MEAT & POULTRY,SHORT_VERTICAL_OEM-INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_OTHER,SHORT_VERTICAL_OTHER FOOD,SHORT_VERTICAL_PACKAGING MATERIALS,SHORT_VERTICAL_PET FOOD & ANIMAL FEED,SHORT_VERTICAL_PHARMA & MEDICAL,SHORT_VERTICAL_POSTAL,SHORT_VERTICAL_SALTY SNACKS,SHORT_VERTICAL_TEXTILE,SHORT_VERTICAL_TOBACCO,SHORT_VERTICAL_UNKNOWN,CUSTOMER_CLASS_END USER,CUSTOMER_CLASS_OEM,TERRITORY_TYPE_Industrial,TERRITORY_TYPE_Postal,SUPPLIES_SEGMENTATION_M,SUPPLIES_SEGMENTATION_S,SUPPLIES_SEGMENTATION_Unclass,SUPPLIES_SEGMENTATION_XL,SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold,SUPPLIES_DECLINE_REASONS_Migration to 1000 Line/TIJ/TTO/LCM/LPA,SUPPLIES_DECLINE_REASONS_Migration to Lasers,SUPPLIES_DECLINE_REASONS_Moved Equipment,SUPPLIES_DECLINE_REASONS_No More Coding Requirement,SUPPLIES_DECLINE_REASONS_None,SUPPLIES_DECLINE_REASONS_Off Brand,SUPPLIES_DECLINE_REASONS_Over Stocked / Timing,SUPPLIES_DECLINE_REASONS_Pricing / Discounting,SUPPLIES_DECLINE_REASONS_Printing/EQ downtime Issues,SUPPLIES_DECLINE_REASONS_Production / Code Reduction,SUPPLIES_DECLINE_REASONS_Production Down (timing),SUPPLIES_DECLINE_REASONS_Project Based,SUPPLIES_DECLINE_REASONS_Recent Regain/Win-back,SUPPLIES_DECLINE_REASONS_Seasonal Producer,SUPPLIES_DECLINE_REASONS_Served by Authorized Distributor,SUPPLIES_DECLINE_REASONS_Site Closed,SUPPLIES_DECLINE_REASONS_VJ Operations Issues,SALES_CHANNEL_Copy,SALES_CHANNEL_ECOMM PO IMPORT,SALES_CHANNEL_EDI,SALES_CHANNEL_Esker,SALES_CHANNEL_IStore Account,SALES_CHANNEL_OCC,SALES_CHANNEL_Online,SALES_CHANNEL_SFDC_CPQ,SALES_CHANNEL_Service Billing,PRODUCT_FAMILY_CIJ,PRODUCT_FAMILY_GRAPHICS,PRODUCT_FAMILY_GRAPHICS BA,PRODUCT_FAMILY_LASER,PRODUCT_FAMILY_LCM,PRODUCT_FAMILY_LPA,PRODUCT_FAMILY_RAW MATERIAL,PRODUCT_FAMILY_TIJ,PRODUCT_FAMILY_TTO,PRODUCT_MODEL_CLEANING SOLUTION,PRODUCT_MODEL_FUME EXTRACTION,PRODUCT_MODEL_INK,PRODUCT_MODEL_LABELS,PRODUCT_MODEL_MAKE-UP,PRODUCT_MODEL_PACKAGING,PRODUCT_MODEL_RIBBONS,PRODUCT_MODEL_SOLVENT,PRODUCT_MODEL_VALUE PACK,Most_Frequent_Interaction_Type_Callback,Most_Frequent_Interaction_Type_Contact Customer,Most_Frequent_Interaction_Type_Customer Meeting,Most_Frequent_Interaction_Type_Dial,Most_Frequent_Interaction_Type_Email,Most_Frequent_Interaction_Type_Make Qualified Sales Call,Most_Frequent_Interaction_Type_Meeting,Most_Frequent_Interaction_Type_Other,Most_Frequent_Interaction_Type_TS Task,Max_Case_Origin_CX Survey Detractor,Max_Case_Origin_Email,Max_Case_Origin_Email - VTI CC Sales Escalations,Max_Case_Origin_Email - VTI NACC,Max_Case_Origin_Email/Fax - VTI CS,Max_Case_Origin_FS Survey Followup,Max_Case_Origin_Install Complete,Max_Case_Origin_Phone,Max_Case_Origin_TS Survey Followup,Max_Case_Origin_unknown,Max_Case_Reason_CX: Customer Care,Max_Case_Reason_CX: Field Sales,Max_Case_Reason_CX: Field Service,Max_Case_Reason_CX: Manufacturing,Max_Case_Reason_CX: Other Team,Max_Case_Reason_CX: Tech Support,Max_Case_Reason_Customer Experience,Max_Case_Reason_unknown,Contract_Category_Full Care,Contract_Category_No Contract,Contract_Category_Supportive,Contract_Category_WFC,TERRITORY_REGION_MW,TERRITORY_REGION_NE,TERRITORY_REGION_NW,TERRITORY_

In [48]:
lcmChurned = lcm[['Churned_365']]

In [49]:
lcm.drop(columns=['Churned_365'],axis=1,inplace=True)

In [50]:
#Scaling
# Instantiate
scaler = StandardScaler()

# fit_transform
lcm_scaled = scaler.fit_transform(lcm)

In [51]:
lcm_scaled = pd.DataFrame(lcm_scaled)
lcm_scaled.columns = lcm.columns
lcm_scaled.head()

,Site_Level_Price_Index,TRX_AMT_USD,Margin,QUANTITY,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Total_Visits,Total_Cases,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,Frequency,Num_of_Trxns,Avg_Margin,Avg_Quantity,Types_of_Product_Family,Types_of_Product_Model,Avg_Price_Index,Tenure,SHORT_VERTICAL_BAKED GOODS & CEREALS,SHORT_VERTICAL_BEVERAGE,SHORT_VERTICAL_BUILDING MATERIALS,SHORT_VERTICAL_CANDY & CONFECTION,SHORT_VERTICAL_CHEMICALS,SHORT_VERTICAL_COSMETICS / PERSONAL CARE,SHORT_VERTICAL_DAIRY & EGGS,SHORT_VERTICAL_DISTRIBUTOR,SHORT_VERTICAL_ELECTRICAL / ELECTRONICS,SHORT_VERTICAL_EXTRUSION / WIRE & CABLE,SHORT_VERTICAL_FISH & SEAFOOD,SHORT_VERTICAL_FROZEN PREPARED MEALS,SHORT_VERTICAL_FRUIT & VEGETABLE,SHORT_VERTICAL_GRAPHICS,SHORT_VERTICAL_INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_MEAT & POULTRY,SHORT_VERTICAL_OEM-INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_OTHER,SHORT_VERTICAL_OTHER FOOD,SHORT_VERTICAL_PACKAGING MATERIALS,SHORT_VERTICAL_PET FOOD & ANIMAL FEED,SHORT_VERTICAL_PHARMA & MEDICAL,SHORT_VERTICAL_POSTAL,SHORT_VERTICAL_SALTY SNACKS,SHORT_VERTICAL_TEXTILE,SHORT_VERTICAL_TOBACCO,SHORT_VERTICAL_UNKNOWN,CUSTOMER_CLASS_END USER,CUSTOMER_CLASS_OEM,TERRITORY_TYPE_Industrial,TERRITORY_TYPE_Postal,SUPPLIES_SEGMENTATION_M,SUPPLIES_SEGMENTATION_S,SUPPLIES_SEGMENTATION_Unclass,SUPPLIES_SEGMENTATION_XL,SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold,SUPPLIES_DECLINE_REASONS_Migration to 1000 Line/TIJ/TTO/LCM/LPA,SUPPLIES_DECLINE_REASONS_Migration to Lasers,SUPPLIES_DECLINE_REASONS_Moved Equipment,SUPPLIES_DECLINE_REASONS_No More Coding Requirement,SUPPLIES_DECLINE_REASONS_None,SUPPLIES_DECLINE_REASONS_Off Brand,SUPPLIES_DECLINE_REASONS_Over Stocked / Timing,SUPPLIES_DECLINE_REASONS_Pricing / Discounting,SUPPLIES_DECLINE_REASONS_Printing/EQ downtime Issues,SUPPLIES_DECLINE_REASONS_Production / Code Reduction,SUPPLIES_DECLINE_REASONS_Production Down (timing),SUPPLIES_DECLINE_REASONS_Project Based,SUPPLIES_DECLINE_REASONS_Recent Regain/Win-back,SUPPLIES_DECLINE_REASONS_Seasonal Producer,SUPPLIES_DECLINE_REASONS_Served by Authorized Distributor,SUPPLIES_DECLINE_REASONS_Site Closed,SUPPLIES_DECLINE_REASONS_VJ Operations Issues,SALES_CHANNEL_Copy,SALES_CHANNEL_ECOMM PO IMPORT,SALES_CHANNEL_EDI,SALES_CHANNEL_Esker,SALES_CHANNEL_IStore Account,SALES_CHANNEL_OCC,SALES_CHANNEL_Online,SALES_CHANNEL_SFDC_CPQ,SALES_CHANNEL_Service Billing,PRODUCT_FAMILY_CIJ,PRODUCT_FAMILY_GRAPHICS,PRODUCT_FAMILY_GRAPHICS BA,PRODUCT_FAMILY_LASER,PRODUCT_FAMILY_LCM,PRODUCT_FAMILY_LPA,PRODUCT_FAMILY_RAW MATERIAL,PRODUCT_FAMILY_TIJ,PRODUCT_FAMILY_TTO,PRODUCT_MODEL_CLEANING SOLUTION,PRODUCT_MODEL_FUME EXTRACTION,PRODUCT_MODEL_INK,PRODUCT_MODEL_LABELS,PRODUCT_MODEL_MAKE-UP,PRODUCT_MODEL_PACKAGING,PRODUCT_MODEL_RIBBONS,PRODUCT_MODEL_SOLVENT,PRODUCT_MODEL_VALUE PACK,Most_Frequent_Interaction_Type_Callback,Most_Frequent_Interaction_Type_Contact Customer,Most_Frequent_Interaction_Type_Customer Meeting,Most_Frequent_Interaction_Type_Dial,Most_Frequent_Interaction_Type_Email,Most_Frequent_Interaction_Type_Make Qualified Sales Call,Most_Frequent_Interaction_Type_Meeting,Most_Frequent_Interaction_Type_Other,Most_Frequent_Interaction_Type_TS Task,Max_Case_Origin_CX Survey Detractor,Max_Case_Origin_Email,Max_Case_Origin_Email - VTI CC Sales Escalations,Max_Case_Origin_Email - VTI NACC,Max_Case_Origin_Email/Fax - VTI CS,Max_Case_Origin_FS Survey Followup,Max_Case_Origin_Install Complete,Max_Case_Origin_Phone,Max_Case_Origin_TS Survey Followup,Max_Case_Origin_unknown,Max_Case_Reason_CX: Customer Care,Max_Case_Reason_CX: Field Sales,Max_Case_Reason_CX: Field Service,Max_Case_Reason_CX: Manufacturing,Max_Case_Reason_CX: Other Team,Max_Case_Reason_CX: Tech Support,Max_Case_Reason_Customer Experience,Max_Case_Reason_unknown,Contract_Category_Full Care,Contract_Category_No Contract,Contract_Category_Supportive,Contract_Category_WFC,TERRITORY_REGION_MW,TERRITORY_REGION_NE,TERRITORY_REGION_NW,TERRITORY_REGION_SC,TE

In [52]:
lcm_final = pd.concat([lcm_scaled,lcmChurned],axis=1,sort=False)

In [53]:
lcm_final = lcm_final.drop(['SUPPLIES_SEGMENTATION_Unclass', 'SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold', 'SUPPLIES_DECLINE_REASONS_Pricing / Discounting', 'PRODUCT_FAMILY_CIJ', 'PRODUCT_FAMILY_GRAPHICS', 'PRODUCT_FAMILY_GRAPHICS BA', 'PRODUCT_FAMILY_LASER', 'PRODUCT_FAMILY_LCM', 'PRODUCT_FAMILY_LPA', 'PRODUCT_FAMILY_RAW MATERIAL', 'PRODUCT_FAMILY_TIJ', 'PRODUCT_FAMILY_TTO', 'PRODUCT_MODEL_CLEANING SOLUTION', 'PRODUCT_MODEL_FUME EXTRACTION', 'PRODUCT_MODEL_LABELS', 'PRODUCT_MODEL_MAKE-UP', 'PRODUCT_MODEL_PACKAGING', 'PRODUCT_MODEL_RIBBONS', 'PRODUCT_MODEL_VALUE PACK', 'Most_Frequent_Interaction_Type_Callback', 'Most_Frequent_Interaction_Type_Dial', 'Most_Frequent_Interaction_Type_TS Task', 'Max_Case_Origin_Email - VTI CC Sales Escalations', 'Max_Case_Reason_CX: Manufacturing', 'Contract_Category_WFC', 'Most_Frequent_Sales_Channel_Service Billing', 'Most_Frequent_Order_Type_DEMO EQUIPMENT ACCEPT', 'Most_Frequent_Order_Type_SERVICE', 'Most_Frequent_Order_Type_US FULL CARE EQPT DOMESTIC', 'Most_Frequent_Order_Type_US FULL CARE INTERNATIONAL'],axis=1)

In [54]:
lcm_final.shape

(15658, 127)

In [55]:
cph2 = lifelines.CoxPHFitter(penalizer=0.00001)
cph2.fit(lcm_final, step_size=0.05, duration_col='Tenure', event_col='Churned_365', show_progress=False)
cph2.print_summary()

<lifelines.CoxPHFitter: fitted with 15658 total observations, 13931 right-censored observations>
             duration col = 'Tenure'
                event col = 'Churned_365'
                penalizer = 1e-05
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 15658
number of events observed = 1727
   partial log-likelihood = -13407.90
         time fit was run = 2020-08-05 04:49:53 UTC

---
                                                                  coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                               
Site_Level_Price_Index                                           -0.05       0.95       0.05            -0.14             0.04                 0.87                 1.04
TRX_AMT_USD                                                       1.16       3.19       0.89            -0.59             2.91                 0.56                18.34
Margin                                                           -0.85       0.43       0.81            -2.43             0.73                 0.09                 2.07
QUANTITY                                                         -0.10       0.90       0.10            -0.31             0.10                 0.74                 1.11
Total_SVC_Incidents                                               0.22       1.24       1.99            -3.68             4.12                 0.03                61.61
Total_Repeat_Calls                                               -0.03       0.98       0.72            -1.43             1.38                 0.24                 3.97
Total_FTF_Calls                                                   0.32       1.38       1.41            -2.44             3.09                 0.09                21.88
Total_Visits                                                     -0.77       0.46       0.11            -0.99            -0.55                 0.37                 0.58
Total_Cases                                                      -0.29       0.75       0.04            -0.36            -0.21                 0.70                 0.81
Num_of_Active_Install_Bases                                      -1.22       0.29       0.21            -1.64            -0.81                 0.19                 0.45
Total_Contracts                                                   0.37       1.45       0.15             0.07             0.66                 1.08                 1.94
Contract_length                                                  -0.67       0.51       0.09            -0.85            -0.48                 0.43                 0.62
Num_of_Inactive_Install_Bases                                     0.01       1.01       0.07            -0.11             0.14                 0.89                 1.15
STRATEGIC_ACCOUNTS                                                0.42       1.53       0.04             0.34             0.51                 1.40                 1.66
Frequency                                                        -0.20       0.82       0.03            -0.25            -0.14                 0.78                 0.87
Num_of_Trxns                                                     -2.41       0.09       0.17            -2.75            -2.07                 0.06                 0.13
Avg_Margin                                                       -0.55       0.58       0.09            -0.72            -0.38                 0.49                 0.68
Avg_Quantity                                                      0.70       2.02       0.10             0.50             0.91                 1.65                 2.48
Types_of_Product_Family                                           0.04       1.04       0.06            -0.07             0.16                 0.93                 1.17
Types_of_Pr

In [56]:
lcm_final_table = cph2.summary[cph2.summary['p']<0.005].sort_values(by='exp(coef)', ascending=False).head(10)

# Model building for TIJ

In [58]:
tij.reset_index(inplace=True,drop=True)

In [59]:
tij.head()

,Site_Level_Price_Index,TRX_AMT_USD,Margin,QUANTITY,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Total_Visits,Total_Cases,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,Frequency,Num_of_Trxns,Avg_Margin,Avg_Quantity,Types_of_Product_Family,Types_of_Product_Model,Avg_Price_Index,Tenure,Churned_365,SHORT_VERTICAL_BAKED GOODS & CEREALS,SHORT_VERTICAL_BEVERAGE,SHORT_VERTICAL_BUILDING MATERIALS,SHORT_VERTICAL_CANDY & CONFECTION,SHORT_VERTICAL_CHEMICALS,SHORT_VERTICAL_COSMETICS / PERSONAL CARE,SHORT_VERTICAL_DAIRY & EGGS,SHORT_VERTICAL_DISTRIBUTOR,SHORT_VERTICAL_ELECTRICAL / ELECTRONICS,SHORT_VERTICAL_EXTRUSION / WIRE & CABLE,SHORT_VERTICAL_FISH & SEAFOOD,SHORT_VERTICAL_FROZEN PREPARED MEALS,SHORT_VERTICAL_FRUIT & VEGETABLE,SHORT_VERTICAL_GRAPHICS,SHORT_VERTICAL_INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_MEAT & POULTRY,SHORT_VERTICAL_OEM-INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_OTHER,SHORT_VERTICAL_OTHER FOOD,SHORT_VERTICAL_PACKAGING MATERIALS,SHORT_VERTICAL_PET FOOD & ANIMAL FEED,SHORT_VERTICAL_PHARMA & MEDICAL,SHORT_VERTICAL_POSTAL,SHORT_VERTICAL_SALTY SNACKS,SHORT_VERTICAL_TEXTILE,SHORT_VERTICAL_TOBACCO,SHORT_VERTICAL_UNKNOWN,CUSTOMER_CLASS_END USER,CUSTOMER_CLASS_OEM,TERRITORY_TYPE_Industrial,TERRITORY_TYPE_Postal,SUPPLIES_SEGMENTATION_M,SUPPLIES_SEGMENTATION_S,SUPPLIES_SEGMENTATION_Unclass,SUPPLIES_SEGMENTATION_XL,SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold,SUPPLIES_DECLINE_REASONS_Migration to 1000 Line/TIJ/TTO/LCM/LPA,SUPPLIES_DECLINE_REASONS_Migration to Lasers,SUPPLIES_DECLINE_REASONS_Moved Equipment,SUPPLIES_DECLINE_REASONS_No More Coding Requirement,SUPPLIES_DECLINE_REASONS_None,SUPPLIES_DECLINE_REASONS_Off Brand,SUPPLIES_DECLINE_REASONS_Over Stocked / Timing,SUPPLIES_DECLINE_REASONS_Pricing / Discounting,SUPPLIES_DECLINE_REASONS_Printing/EQ downtime Issues,SUPPLIES_DECLINE_REASONS_Production / Code Reduction,SUPPLIES_DECLINE_REASONS_Production Down (timing),SUPPLIES_DECLINE_REASONS_Project Based,SUPPLIES_DECLINE_REASONS_Recent Regain/Win-back,SUPPLIES_DECLINE_REASONS_Seasonal Producer,SUPPLIES_DECLINE_REASONS_Served by Authorized Distributor,SUPPLIES_DECLINE_REASONS_Site Closed,SUPPLIES_DECLINE_REASONS_VJ Operations Issues,SALES_CHANNEL_Copy,SALES_CHANNEL_ECOMM PO IMPORT,SALES_CHANNEL_EDI,SALES_CHANNEL_Esker,SALES_CHANNEL_IStore Account,SALES_CHANNEL_OCC,SALES_CHANNEL_Online,SALES_CHANNEL_SFDC_CPQ,SALES_CHANNEL_Service Billing,PRODUCT_FAMILY_CIJ,PRODUCT_FAMILY_GRAPHICS,PRODUCT_FAMILY_GRAPHICS BA,PRODUCT_FAMILY_LASER,PRODUCT_FAMILY_LCM,PRODUCT_FAMILY_LPA,PRODUCT_FAMILY_RAW MATERIAL,PRODUCT_FAMILY_TIJ,PRODUCT_FAMILY_TTO,PRODUCT_MODEL_CLEANING SOLUTION,PRODUCT_MODEL_FUME EXTRACTION,PRODUCT_MODEL_INK,PRODUCT_MODEL_LABELS,PRODUCT_MODEL_MAKE-UP,PRODUCT_MODEL_PACKAGING,PRODUCT_MODEL_RIBBONS,PRODUCT_MODEL_SOLVENT,PRODUCT_MODEL_VALUE PACK,Most_Frequent_Interaction_Type_Callback,Most_Frequent_Interaction_Type_Contact Customer,Most_Frequent_Interaction_Type_Customer Meeting,Most_Frequent_Interaction_Type_Dial,Most_Frequent_Interaction_Type_Email,Most_Frequent_Interaction_Type_Make Qualified Sales Call,Most_Frequent_Interaction_Type_Meeting,Most_Frequent_Interaction_Type_Other,Most_Frequent_Interaction_Type_TS Task,Max_Case_Origin_CX Survey Detractor,Max_Case_Origin_Email,Max_Case_Origin_Email - VTI CC Sales Escalations,Max_Case_Origin_Email - VTI NACC,Max_Case_Origin_Email/Fax - VTI CS,Max_Case_Origin_FS Survey Followup,Max_Case_Origin_Install Complete,Max_Case_Origin_Phone,Max_Case_Origin_TS Survey Followup,Max_Case_Origin_unknown,Max_Case_Reason_CX: Customer Care,Max_Case_Reason_CX: Field Sales,Max_Case_Reason_CX: Field Service,Max_Case_Reason_CX: Manufacturing,Max_Case_Reason_CX: Other Team,Max_Case_Reason_CX: Tech Support,Max_Case_Reason_Customer Experience,Max_Case_Reason_unknown,Contract_Category_Full Care,Contract_Category_No Contract,Contract_Category_Supportive,Contract_Category_WFC,TERRITORY_REGION_MW,TERRITORY_REGION_NE,TERRITORY_REGION_NW,TERRITORY_

In [60]:
tijChurned = tij[['Churned_365']]

In [61]:
tij.drop(columns=['Churned_365'],axis=1,inplace=True)

In [62]:
#Scaling
# Instantiate
scaler = StandardScaler()

# fit_transform
tij_scaled = scaler.fit_transform(tij)

In [63]:
tij_scaled = pd.DataFrame(tij_scaled)
tij_scaled.columns = tij.columns
tij_scaled.head()

,Site_Level_Price_Index,TRX_AMT_USD,Margin,QUANTITY,Total_SVC_Incidents,Total_Repeat_Calls,Total_FTF_Calls,Total_Visits,Total_Cases,Num_of_Active_Install_Bases,Total_Contracts,Contract_length,Num_of_Inactive_Install_Bases,STRATEGIC_ACCOUNTS,Frequency,Num_of_Trxns,Avg_Margin,Avg_Quantity,Types_of_Product_Family,Types_of_Product_Model,Avg_Price_Index,Tenure,SHORT_VERTICAL_BAKED GOODS & CEREALS,SHORT_VERTICAL_BEVERAGE,SHORT_VERTICAL_BUILDING MATERIALS,SHORT_VERTICAL_CANDY & CONFECTION,SHORT_VERTICAL_CHEMICALS,SHORT_VERTICAL_COSMETICS / PERSONAL CARE,SHORT_VERTICAL_DAIRY & EGGS,SHORT_VERTICAL_DISTRIBUTOR,SHORT_VERTICAL_ELECTRICAL / ELECTRONICS,SHORT_VERTICAL_EXTRUSION / WIRE & CABLE,SHORT_VERTICAL_FISH & SEAFOOD,SHORT_VERTICAL_FROZEN PREPARED MEALS,SHORT_VERTICAL_FRUIT & VEGETABLE,SHORT_VERTICAL_GRAPHICS,SHORT_VERTICAL_INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_MEAT & POULTRY,SHORT_VERTICAL_OEM-INDUSTRIAL EQUIPMENT,SHORT_VERTICAL_OTHER,SHORT_VERTICAL_OTHER FOOD,SHORT_VERTICAL_PACKAGING MATERIALS,SHORT_VERTICAL_PET FOOD & ANIMAL FEED,SHORT_VERTICAL_PHARMA & MEDICAL,SHORT_VERTICAL_POSTAL,SHORT_VERTICAL_SALTY SNACKS,SHORT_VERTICAL_TEXTILE,SHORT_VERTICAL_TOBACCO,SHORT_VERTICAL_UNKNOWN,CUSTOMER_CLASS_END USER,CUSTOMER_CLASS_OEM,TERRITORY_TYPE_Industrial,TERRITORY_TYPE_Postal,SUPPLIES_SEGMENTATION_M,SUPPLIES_SEGMENTATION_S,SUPPLIES_SEGMENTATION_Unclass,SUPPLIES_SEGMENTATION_XL,SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold,SUPPLIES_DECLINE_REASONS_Migration to 1000 Line/TIJ/TTO/LCM/LPA,SUPPLIES_DECLINE_REASONS_Migration to Lasers,SUPPLIES_DECLINE_REASONS_Moved Equipment,SUPPLIES_DECLINE_REASONS_No More Coding Requirement,SUPPLIES_DECLINE_REASONS_None,SUPPLIES_DECLINE_REASONS_Off Brand,SUPPLIES_DECLINE_REASONS_Over Stocked / Timing,SUPPLIES_DECLINE_REASONS_Pricing / Discounting,SUPPLIES_DECLINE_REASONS_Printing/EQ downtime Issues,SUPPLIES_DECLINE_REASONS_Production / Code Reduction,SUPPLIES_DECLINE_REASONS_Production Down (timing),SUPPLIES_DECLINE_REASONS_Project Based,SUPPLIES_DECLINE_REASONS_Recent Regain/Win-back,SUPPLIES_DECLINE_REASONS_Seasonal Producer,SUPPLIES_DECLINE_REASONS_Served by Authorized Distributor,SUPPLIES_DECLINE_REASONS_Site Closed,SUPPLIES_DECLINE_REASONS_VJ Operations Issues,SALES_CHANNEL_Copy,SALES_CHANNEL_ECOMM PO IMPORT,SALES_CHANNEL_EDI,SALES_CHANNEL_Esker,SALES_CHANNEL_IStore Account,SALES_CHANNEL_OCC,SALES_CHANNEL_Online,SALES_CHANNEL_SFDC_CPQ,SALES_CHANNEL_Service Billing,PRODUCT_FAMILY_CIJ,PRODUCT_FAMILY_GRAPHICS,PRODUCT_FAMILY_GRAPHICS BA,PRODUCT_FAMILY_LASER,PRODUCT_FAMILY_LCM,PRODUCT_FAMILY_LPA,PRODUCT_FAMILY_RAW MATERIAL,PRODUCT_FAMILY_TIJ,PRODUCT_FAMILY_TTO,PRODUCT_MODEL_CLEANING SOLUTION,PRODUCT_MODEL_FUME EXTRACTION,PRODUCT_MODEL_INK,PRODUCT_MODEL_LABELS,PRODUCT_MODEL_MAKE-UP,PRODUCT_MODEL_PACKAGING,PRODUCT_MODEL_RIBBONS,PRODUCT_MODEL_SOLVENT,PRODUCT_MODEL_VALUE PACK,Most_Frequent_Interaction_Type_Callback,Most_Frequent_Interaction_Type_Contact Customer,Most_Frequent_Interaction_Type_Customer Meeting,Most_Frequent_Interaction_Type_Dial,Most_Frequent_Interaction_Type_Email,Most_Frequent_Interaction_Type_Make Qualified Sales Call,Most_Frequent_Interaction_Type_Meeting,Most_Frequent_Interaction_Type_Other,Most_Frequent_Interaction_Type_TS Task,Max_Case_Origin_CX Survey Detractor,Max_Case_Origin_Email,Max_Case_Origin_Email - VTI CC Sales Escalations,Max_Case_Origin_Email - VTI NACC,Max_Case_Origin_Email/Fax - VTI CS,Max_Case_Origin_FS Survey Followup,Max_Case_Origin_Install Complete,Max_Case_Origin_Phone,Max_Case_Origin_TS Survey Followup,Max_Case_Origin_unknown,Max_Case_Reason_CX: Customer Care,Max_Case_Reason_CX: Field Sales,Max_Case_Reason_CX: Field Service,Max_Case_Reason_CX: Manufacturing,Max_Case_Reason_CX: Other Team,Max_Case_Reason_CX: Tech Support,Max_Case_Reason_Customer Experience,Max_Case_Reason_unknown,Contract_Category_Full Care,Contract_Category_No Contract,Contract_Category_Supportive,Contract_Category_WFC,TERRITORY_REGION_MW,TERRITORY_REGION_NE,TERRITORY_REGION_NW,TERRITORY_REGION_SC,TE

In [64]:
tij_final = pd.concat([tij_scaled,tijChurned],axis=1,sort=False)

In [65]:
tij_final = tij_final.drop(['SHORT_VERTICAL_DISTRIBUTOR', 'SHORT_VERTICAL_EXTRUSION / WIRE & CABLE', 'SHORT_VERTICAL_POSTAL', 'SHORT_VERTICAL_SALTY SNACKS', 'SUPPLIES_SEGMENTATION_Unclass', 'SUPPLIES_DECLINE_REASONS_Financial Distress/Credit Hold', 'SUPPLIES_DECLINE_REASONS_No More Coding Requirement', 'SUPPLIES_DECLINE_REASONS_Pricing / Discounting', 'SUPPLIES_DECLINE_REASONS_Printing/EQ downtime Issues', 'SUPPLIES_DECLINE_REASONS_Seasonal Producer', 'SUPPLIES_DECLINE_REASONS_VJ Operations Issues', 'SALES_CHANNEL_ECOMM PO IMPORT', 'SALES_CHANNEL_Service Billing', 'PRODUCT_FAMILY_CIJ', 'PRODUCT_FAMILY_GRAPHICS', 'PRODUCT_FAMILY_GRAPHICS BA', 'PRODUCT_FAMILY_LASER', 'PRODUCT_FAMILY_LCM', 'PRODUCT_FAMILY_LPA', 'PRODUCT_FAMILY_RAW MATERIAL', 'PRODUCT_FAMILY_TIJ', 'PRODUCT_FAMILY_TTO', 'PRODUCT_MODEL_CLEANING SOLUTION', 'PRODUCT_MODEL_FUME EXTRACTION', 'PRODUCT_MODEL_INK', 'PRODUCT_MODEL_LABELS', 'PRODUCT_MODEL_MAKE-UP', 'PRODUCT_MODEL_PACKAGING', 'PRODUCT_MODEL_RIBBONS', 'PRODUCT_MODEL_SOLVENT', 'PRODUCT_MODEL_VALUE PACK', 'Most_Frequent_Interaction_Type_Callback', 'Most_Frequent_Interaction_Type_Contact Customer', 'Most_Frequent_Interaction_Type_Dial', 'Most_Frequent_Interaction_Type_Make Qualified Sales Call', 'Most_Frequent_Interaction_Type_TS Task', 'Max_Case_Origin_CX Survey Detractor', 'Max_Case_Origin_Email', 'Max_Case_Origin_Email - VTI CC Sales Escalations', 'Max_Case_Origin_FS Survey Followup', 'Max_Case_Reason_CX: Manufacturing', 'Max_Case_Reason_CX: Other Team', 'Contract_Category_Supportive', 'Contract_Category_WFC', 'Most_Frequent_Sales_Channel_Service Billing', 'Most_Frequent_Order_Type_BILL ONLY', 'Most_Frequent_Order_Type_DEMO EQUIPMENT ACCEPT', 'Most_Frequent_Order_Type_SERVICE', 'Most_Frequent_Order_Type_STANDARD INTERNATIONAL', 'Most_Frequent_Order_Type_US FULL CARE EQPT DOMESTIC', 'Most_Frequent_Order_Type_US FULL CARE INTERNATIONAL'
                            ],axis=1)

In [66]:
tij_final.shape

(4395, 106)

In [67]:
cph3 = lifelines.CoxPHFitter(penalizer=0.1)
cph3.fit(tij_final, step_size=0.05, duration_col='Tenure', event_col='Churned_365', show_progress=False)
cph3.print_summary()

<lifelines.CoxPHFitter: fitted with 4395 total observations, 3812 right-censored observations>
             duration col = 'Tenure'
                event col = 'Churned_365'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 4395
number of events observed = 583
   partial log-likelihood = -4027.80
         time fit was run = 2020-08-05 04:51:06 UTC

---
                                                                  coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                                               
Site_Level_Price_Index                                           -0.03       0.97       0.04            -0.11             0.04                 0.90                 1.04
TRX_AMT_USD                                                      -0.03       0.97       0.04            -0.12             0.05                 0.89                 1.05
Margin                                                           -0.04       0.96       0.04            -0.12             0.04                 0.89                 1.04
QUANTITY                                                         -0.01       0.99       0.04            -0.10             0.07                 0.90                 1.07
Total_SVC_Incidents                                               0.02       1.02       0.04            -0.07             0.10                 0.93                 1.11
Total_Repeat_Calls                                                0.05       1.05       0.04            -0.04             0.13                 0.97                 1.14
Total_FTF_Calls                                                  -0.00       1.00       0.04            -0.09             0.09                 0.92                 1.09
Total_Visits                                                     -0.13       0.88       0.04            -0.21            -0.04                 0.81                 0.96
Total_Cases                                                       0.03       1.03       0.03            -0.02             0.09                 0.98                 1.09
Num_of_Active_Install_Bases                                      -0.17       0.84       0.04            -0.26            -0.08                 0.77                 0.92
Total_Contracts                                                  -0.09       0.91       0.05            -0.18            -0.00                 0.84                 1.00
Contract_length                                                  -0.05       0.95       0.04            -0.13             0.02                 0.88                 1.02
Num_of_Inactive_Install_Bases                                    -0.03       0.97       0.04            -0.12             0.06                 0.89                 1.06
STRATEGIC_ACCOUNTS                                               -0.01       0.99       0.04            -0.09             0.07                 0.91                 1.07
Frequency                                                        -0.12       0.88       0.03            -0.18            -0.07                 0.83                 0.94
Num_of_Trxns                                                     -0.18       0.83       0.04            -0.27            -0.09                 0.76                 0.91
Avg_Margin                                                       -0.11       0.90       0.04            -0.19            -0.02                 0.83                 0.98
Avg_Quantity                                                     -0.05       0.95       0.04            -0.13             0.04                 0.88                 1.04
Types_of_Product_Family                                          -0.09       0.91       0.04            -0.17            -0.01                 0.85                 0.99
Types_of_Product_M

In [68]:
tij_final_table = cph3.summary[cph3.summary['p']<0.005].sort_values(by='exp(coef)', ascending=False).head(10)

# Exporting Results

In [69]:
cij_final_table.to_csv("cij_significant_factors.csv")
tto_final_table.to_csv("tto_significant_factors.csv")
lcm_final_table.to_csv("lcm_significant_factors.csv")
tij_final_table.to_csv("tij_significant_factors.csv")